Here we will just create a basic model with the data we have.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import scipy as sc
import xgboost as xgb
import itertools

In [2]:
from xgboost import XGBClassifier

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
data = pd.read_csv('web_dev_segmentation.csv')

In [5]:
data.head()

,customer,project,total_hours,discount%,weekday,hour,num_projects,Frontend%,Backend%,UI/UX%,DevOps%,Maintenance%,Testing%,Consulting%,Security%,class,labels
0,0,0,34.97,10.49,2,14,22.0,0.32,24.53,5.00,18.72,11.79,8.47,11.30,19.86,enterprise_clients,9
1,1,1,28.62,9.68,4,13,4.0,8.78,9.41,11.62,8.88,0.37,17.94,17.31,25.68,mixed_services,3
2,2,2,36.48,14.76,2,22,2.0,14.95,1.67,18.11,11.10,12.05,6.66,19.57,15.88,mixed_services,2
3,3,3,45.23,17.66,5,20,2.0,23.82,7.35,9.91,8.28,19.82,7.85,11.57,11.39,mixed_services,2
4,4,4,27.66,13.43,6,18,1.0,3.60,17.41,16.96,14.03,9.73,19.67,10.12,8.49,mixed_services,0


In [6]:
data.describe()

,customer,project,total_hours,discount%,weekday,hour,num_projects,Frontend%,Backend%,UI/UX%,DevOps%,Maintenance%,Testing%,Consulting%,Security%,labels
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000
mean,24999.500000,24999.500000,30.017719,10.051385,3.996260,15.518120,10.040240,12.514764,12.435788,12.531118,12.550743,12.490497,12.466345,12.487510,12.523213,4.50000
std,14433.901067,14433.901067,9.939912,4.904215,1.998501,4.611817,9.674036,8.690937,8.655000,8.702468,8.687924,8.681824,8.706750,8.678014,8.705430,2.87231
min,0.000000,0.000000,5.000000,0.000000,1.000000,8.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,12499.750000,12499.750000,23.210000,6.650000,2.000000,12.000000,3.000000,5.890000,5.810000,5.890000,5.840000,5.850000,5.820000,5.880000,5.840000,2.00000
50%,24999.500000,24999.500000,30.020000,10.020000,4.000000,16.000000,7.000000,12.060000,11.990000,12.070000,12.130000,12.110000,12.050000,12.080000,12.100000,4.50000
75%,37499.250000,37499.250000,36.770000,13.380000,6.000000,19.000000,14.000000,18.172500,18.020000,18.150000,18.280000,18.140000,18.090000,18.080000,18.170000,7.00000
max,49999.000000,49999.000000,74.790000,25.000000,7.000000,23.000000,50.000000,71.010000,100.000000,73.070000,76.480000,93.830000,73.280000,80.480000,68.140000,9.00000


In [7]:
y = data['labels'].values
X = data.drop(['weekday', 'hour','labels', 'class'], axis=1).values

In [8]:
from sklearn.model_selection import StratifiedKFold

In [9]:
skf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=42)

In [10]:
xgbc = XGBClassifier(objective='multi:softprob', random_state=42)

In [11]:
cv_scores = []
for train_index, test_index in skf.split(X,y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    xgbc.fit(X_train, y_train)
    score = xgbc.score(X_test, y_test)
    print(score)
    cv_scores.append(score)

0.9921
0.992
0.9912
0.9933
0.9914


That's not a bad score, let's try to improve it by tuning parameters.

In [12]:
from sklearn.model_selection import GridSearchCV

In [13]:
params = {
    'max_depth': [6, 7],
    'learning_rate': [0.05],
    'n_estimators': [500],
    'gamma': [0],
    'max_delta_step': [1],
    'subsample': [0.9, 0.8],
    'colsample_bytree': [1.0],
    'colsample_bylevel': [1.0],
    'min_child_weight': [1.0]
}

In [14]:
base_model = XGBClassifier(
    objective='multi:softprob',
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss',
    tree_method='hist'  # Recommended for most scenarios
)

In [15]:
grid_search_xgb = GridSearchCV(
    estimator=base_model,
    param_grid=params,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    n_jobs=-1,
    verbose=1
)

In [16]:
grid_search_xgb.fit(X, y)
print("Best parameters:", grid_search_xgb.best_params_)
print("Best cross-validation score:", grid_search_xgb.best_score_)


Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best parameters: {'colsample_bylevel': 1.0, 'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.05, 'max_delta_step': 1, 'max_depth': 6, 'min_child_weight': 1.0, 'n_estimators': 500, 'subsample': 0.9}
Best cross-validation score: 0.9924800000000001


In [17]:
results = pd.DataFrame(grid_search_xgb.cv_results_)
results = pd.concat([results, pd.DataFrame(grid_search_xgb.cv_results_)], axis=0)

In [18]:
results[results['mean_test_score']==results['mean_test_score'].max()].T

,0,0
mean_fit_time,81.554574,81.554574
std_fit_time,4.167276,4.167276
mean_score_time,2.058414,2.058414
std_score_time,0.140242,0.140242
param_colsample_bylevel,1.0,1.0
param_colsample_bytree,1.0,1.0
param_gamma,0,0
param_learning_rate,0.05,0.05
param_max_delta_step,1,1
param_max_depth,6,6


In [19]:
selected_xgbc = XGBClassifier(learning_rate=0.05, max_depth=6, n_estimators=500, subsample=0.9)

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)

In [22]:
selected_xgbc.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [23]:
selected_xgbc.score(X_train, y_train)

1.0

In [24]:
selected_xgbc.score(X_test, y_test)

0.9942

Well, That's an improvement! This model can be further used to predict the classes of customers.